In [1]:
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import cifar10

In [2]:
import my_image

Датасет состоит из 32 тысяч изображений 20x20 пикселей, 12 тысяч из которых содержат изображение самолета со спутника, а на остальных имеются различные объекты и части самолетов. Датасет представляет собой массив пикселей, каждый из которых представляется в виде точки в трехмерном пространстве RGB. Каждому изображению соответствует число характеризующее класс, к которому оно принадлежит

In [3]:
df = pd.read_json('planesnet.json')
test = df.copy()
df = df.sample(frac=1)

Отбрасываем ненужные столбцы(data-столбец изображений,labels-столбец класса):

In [4]:
df = df[['data', 'labels']]

Нормализуем данные и переводим в матрицу размерности (32000,20,20,3)

Нормализация необходима для корректной работы нейронной сети, так как широкий спектр значений может негативно сказываться на ее работе

In [5]:
data = list(df.data.apply(my_image.image_processing))

In [6]:
data = np.array(data).reshape((32000, 20, 20, 3))

Метки класса приводим в приемлемый вид:

In [7]:
labels = np.array(df.labels.apply(lambda x: x))

Разбиваем данные на обучающую и тестовую выборку в соотношении 20000:12000:

In [8]:
X_train = np.array(data[:20000])
y_train = labels[:20000].reshape((20000, 1))
X_test = np.array(data[20000:])
y_test = labels[20000:].reshape((12000, 1))

Переводим метки классов в бинарный вид:

In [9]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

#Присваиваем переменной значение числа классов в данных
#В нашем случае 2 состояния: есть самолет или его нет
num_classes = y_train.shape[1]

In [10]:
del data

In [4]:
#Создаем модель сверточной нейронной сети на Keras:

#На этапе проектирования сверточной нейронной сети CNN нам необходимо 
#выбрать формат для модели CNN, наиболее часто используется Sequental:

model = Sequential()


'''
Первый слой нашей модели - это сверточный слой. Он будет принимать входные данные 
и пропускать их через сверточные фильтры.
При реализации этого в Keras, мы должны указать количество каналов (фильтров), 
которое нам нужно (а это 32), размер фильтра (3 x 3 в нашем случае), 
форму входа (при создании первого слоя), функцию активации и отступы.
Как уже упоминалось, relu является наиболее распространенной функцией активации, 
а отступы мы определим через padding = 'same', то есть, мы не меняем размер изображения:
'''

model.add(Conv2D(32, (3, 3), input_shape=(20, 20, 3), padding='same'))
model.add(Activation('relu'))

'''
Теперь мы создадим исключающий слой для предотвращения переобучения,
который случайным образом устраняет соединения между слоями 
(0,2 означает, что он отбрасывает 20% существующих соединений):
'''

model.add(Dropout(0.2))

'''
Также мы можем выполнить пакетную нормализацию. Пакетная нормализация нормализует входные 
данные, поступающие в следующий слой, гарантируя, что сеть всегда создает 
функции активации с тем же распределением, которое нам нужно:
'''

model.add(BatchNormalization())

'''
Теперь следует еще один сверточный слой, но размер фильтра увеличивается, 
так что сеть уже может изучать более сложные представления:
'''

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

'''
А вот и объединяющий слой, который, как обсуждалось ранее, помогает сделать 
классификатор изображений более корректным, чтобы он мог изучать релевантные шаблоны.
Также опишем исключение (Dropout) и пакетную нормализацию:
'''

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

'''
Необходимое количество объединяющих слоев зависит от выполняемой задачи. 
Поскольку изображения в нашем наборе уже достаточно малы, мы не будем объединять 
их более двух раз.Теперь можно повторить эти слои, 
чтобы дать сети больше представлений для работы:
'''

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

'''
После того, как мы закончили со сверточными слоями, нам нужно сжать данные, 
поэтому мы импортировали функцию Flatten выше. Мы также добавим слой исключения снова:
'''

model.add(Flatten())
model.add(Dropout(0.2))

'''
Теперь мы используем импортированную функцию Dense и создаем первый плотно связанный слой.
Нам нужно указать количество нейронов в плотном слое. Число нейронов в последующих 
слоях уменьшается, в конечном итоге приближаясь к тому же числу нейронов, что и классы 
в наборе данных (в данном случае 2). Ограничение ядра может упорядочить данные 
в процессе обучения, что также помогает предотвратить переобучение. 
Вот почему мы импортировали maxnorm ранее.
'''

model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

'''
В этом последнем слое мы уравниваем количество классов с числом нейронов, каждый 
из которых хранит некоторую вероятность того, что рассматриваемое изображение 
принадлежит его классу. Наконец, функция активации softmax выбирает нейрон с 
наибольшей вероятностью в качестве своего выходного значения, предполагая, 
что изображение принадлежит именно этому классу:
'''

model.add(Dense(num_classes))
model.add(Activation('softmax'))

'''
Теперь, когда мы разработали модель, которую хотим использовать, остаётся лишь 
скомпилировать ее. Укажем количество эпох для обучения(эмпирическим путем было 
подобрано оптимальное количество эпох обучения) , а также оптимизатор, 
который мы хотим использовать.
Оптимизатор - это то, что настроит веса в вашей сети так, чтобы приблизиться 
к точке с наименьшими потерями. Алгоритм Адама является одним из наиболее 
часто используемых оптимизаторов, потому что он дает высокую 
производительность в большинстве задач:
'''

epochs = 9
optimizer = 'Adam'

#Теперь скомпилируем модель с выбранными параметрами. Также укажем метрику для оценки.

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

print(model.summary())

'''
Теперь мы приступаем к обучению модели. Для этого нам нужно вызвать функцию fit() 
для модели и передать выбранные параметры.
'''

model.fit(X_train,
          y_train,
          validation_data=(X_test, y_test),
          epochs=epochs,
          batch_size=64)

'''
Теперь мы можем оценить модель и посмотреть, как она работает. 
Просто вызовем model.evaluate():
'''

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

NameError: name 'Sequential' is not defined

In [12]:
#Сохраняем  обученную модель, чтобы потом импортировать её при необходимости

model.save('plane_recognition.h5')

In [13]:
del X_train, X_test, y_train, y_test

In [14]:
#Опишем метод для более удобного представления результатов распознавания:

def prediction(data):
    predict = model.predict(data)
    predict = np.round(predict)
    for input_data in predict:
        if input_data[0] == 0 and input_data[1] == 1:
            return 'Самолет найден'
    return 'Самолет отсутствует'

In [15]:
'''
Применяем этот метод для папки ‘conturs’, 
в которой находится набор снимков спутника с самолетами:
'''

files = os.listdir('contours')
for file in files:
    path = os.path.join('contours', file)
    print(path)
    print(prediction(my_image.search_image(path)))

conturs/3.jpg
The original image size is 749 wide x 804 high
The resized image size is 400 wide x 400 high
Самолет отсутствует
conturs/10.jpg
The original image size is 1127 wide x 789 high
The resized image size is 400 wide x 400 high
Самолет найден
conturs/1.jpg
The original image size is 1027 wide x 662 high
The resized image size is 400 wide x 400 high
Самолет найден
conturs/4.jpg
The original image size is 752 wide x 737 high
The resized image size is 400 wide x 400 high
Самолет найден
conturs/8.jpg
The original image size is 1249 wide x 898 high
The resized image size is 400 wide x 400 high
Самолет найден
conturs/2.jpg
The original image size is 1022 wide x 663 high
The resized image size is 400 wide x 400 high
Самолет найден
conturs/11.jpeg
The original image size is 259 wide x 194 high
The resized image size is 400 wide x 400 high
Самолет отсутствует
conturs/6.jpg
The original image size is 754 wide x 1008 high
The resized image size is 400 wide x 400 high
Самолет найден
contur